In [1]:
# import library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import seaborn as sns
from IPython.display import display

import pingouin as pg
import researchpy   
%matplotlib inline

In [2]:
data = pd.read_csv("Titanic_train.csv")
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [3]:
#Q1: 透過數值法計算 Age 和 Survived 是否有相關性?
#turn Survived into discrete
data['Survived'] = data['Survived'].apply(lambda x: 'no' if x==0 else 'yes')
display(data)
#!pip install pingouin
import pingouin as pg
#pg.anova 計算變異數
aov = pg.anova(dv='Age', between='Survived', data=data, detailed=True)
aov

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,no,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,yes,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,yes,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,yes,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,no,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,no,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,yes,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,no,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,yes,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


,Source,SS,DF,MS,F,p-unc,np2
0,Survived,897.187582,1,897.187582,4.271195,0.039125,0.005963
1,Within,149559.448362,712,210.055405,NaN,NaN,NaN


In [4]:
#計算相關性 Eta Squared (η2)
etaSq = aov.SS[0] / (aov.SS[0] + aov.SS[1])
etaSq

0.0059630974469252

In [5]:
def judgment_etaSq(etaSq):
    if etaSq < .01:
        qual = 'Negligible'
    elif etaSq < .06:
        qual = 'Small'
    elif etaSq < .14:
        qual = 'Medium'
    else:
        qual = 'Large'
    return(qual)
judgment_etaSq(etaSq)

'Negligible'

In [6]:
#Q2:透過數值法計算 Sex 和 Survived 是否有相關性?
#Step 1：用交叉列連表(contingency table) 整理資料
contTable = pd.crosstab(data['Sex'], data['Survived'])
contTable

Survived,no,yes
Sex,,
female,81,233
male,468,109


In [7]:
#Step 2：計算資料自由度 df
df = min(contTable.shape[0], contTable.shape[1]) - 1
df

1

In [16]:
#Step 3：運用 researchpy 套件，計算出 Cramer’s V 係數
crosstab, res = researchpy.crosstab(data['Sex'], data['Survived'], test='chi-square')
#print(res)
print("Cramer's value is",res.loc[2,'results'])
def judgment_cramer(cramer_value):
    if cramer_value < 0.1:
        qual = 'Negligible'
    elif cramer_value < 0.3:
        qual = 'Small'
    elif cramer_value < 0.5:
        qual = 'Medium'
    else:
        qual = 'Large'
    return(qual)
judgment_cramer(res.loc[2,'results'])
#這邊用卡方檢定獨立性，所以採用的 test 參數為卡方 "test =" argument.
# 採用的變數在這個模組中，會自己根據資料集來判斷，Cramer's Phi if it a 2x2 table, or Cramer's V is larger than 2x2.

Cramer's value is 0.5434


'Large'

In [9]:
#Q3: 透過數值法計算 Age 和 Fare 是否有相關性?


#Step0:幫Age填補缺失值
#Step1：離散轉連續型資料
#讓 Male=1, FeMale-0.

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data['Sex']=le.fit_transform(data['Sex'])
print(data['Sex'])
data_2=data[['Age','Pclass','Sex','Fare']]

from sklearn.metrics.pairwise import nan_euclidean_distances
display(pd.DataFrame(nan_euclidean_distances(data_2)))
# KNN
# 設定 k 值
value_neighbors=1
from sklearn.impute import KNNImputer
#初始化: Initialize KNNImputer
imputer = KNNImputer(n_neighbors=value_neighbors)
df_filled = pd.DataFrame(imputer.fit_transform(data_2))
#display(data[missing_index.values==True])
#display(df_filled[missing_index.values==True])
display(df_filled)

0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: Sex, Length: 891, dtype: int32


,0,1,2,3,4,5,6,7,8,9,...,881,882,883,884,885,886,887,888,889,890
0,0.000000,66.039863,4.177993,47.709774,13.024592,1.395225,54.938831,24.313178,6.409370,24.223726,...,11.018941,3.416333,6.896557,3.006659,27.722114,7.685213,23.055639,18.741754,23.185394,10.012492
1,66.039863,0.000000,64.515689,18.429118,63.343904,72.589996,25.182682,61.821302,61.180246,47.701888,...,63.623700,62.869545,61.616634,65.573751,42.217559,59.329108,45.445691,55.281396,43.003614,63.855150
2,4.177993,64.515689,0.000000,46.106189,9.056248,1.308642,52.148863,27.384713,3.360534,25.207865,...,7.071128,4.766226,3.553959,1.663017,24.868454,5.362427,23.244475,17.926726,22.187961,6.085279
3,47.709774,18.429118,46.106189,0.000000,45.105460,51.612419,19.066500,46.039121,42.769193,31.182432,...,45.303639,44.568346,43.194444,47.176292,24.388535,40.914667,28.100000,34.314671,24.811489,45.504093
4,13.024592,63.343904,9.056248,45.105460,0.000000,0.471464,47.796811,35.477466,8.631729,30.461708,...,2.005936,13.269688,7.483482,10.049876,21.474534,9.460576,27.254403,17.819839,23.807614,3.014963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,7.685213,59.329108,5.362427,40.914667,9.460576,5.369921,47.331743,26.290790,2.341916,21.480508,...,7.940583,5.759061,2.692582,6.356296,20.149829,0.000000,18.841444,12.176617,17.058722,7.318641
887,23.055639,45.445691,23.244475,28.100000,27.254403,25.007861,41.279158,19.330174,20.590104,5.099511,...,26.260153,19.814111,21.523243,23.826508,20.118788,18.841444,0.000000,7.908013,7.071068,25.866242
888,18.741754,55.281396,17.926726,34.314671,17.819839,17.349393,32.909374,2.975595,14.222100,7.731752,...,17.997523,14.934088,15.042274,18.972260,6.552926,12.176617,7.908013,0.000000,7.991871,18.165535
889,23.185394,43.003614,22.187961,24.811489,23.807614,24.981189,35.524202,25.683762,19.025046,12.083253,...,23.272208,20.014969,19.627786,23.058675,13.219895,17.058722,7.071068,7.991871,0.000000,23.131418


,0,1,2,3
0,22.0,3.0,1.0,7.2500
1,38.0,1.0,0.0,71.2833
2,26.0,3.0,0.0,7.9250
3,35.0,1.0,0.0,53.1000
4,35.0,3.0,1.0,8.0500
...,...,...,...,...
886,27.0,2.0,1.0,13.0000
887,19.0,1.0,0.0,30.0000
888,30.0,3.0,0.0,23.4500
889,26.0,1.0,1.0,30.0000


In [14]:
# 由於 pearsonr 有兩個回傳結果，我們只需取第一個回傳值為相關係數
corr, _=stats.pearsonr(df_filled[0], df_filled[3])
print(corr)

def judgment_corr(corr):
    if corr < 0.1:
        qual = 'Negligible'
    elif corr < 0.39:
        qual = 'Small'
    elif corr < 0.69:
        qual = 'Medium'
    elif corr < 0.99:
        qual = 'Large'
    else:
        qual = 'Total'
    return(qual)
judgment_corr(corr)

0.08211205639457379


'Negligible'